In [ ]:
!pip install tensorflow==2.6.0
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import (Dense,Conv2D,MaxPool2D,Flatten,Dropout,BatchNormalization)
from tensorflow.keras.preprocessing import image as image_utils
from matplotlib import pyplot as plt
import numpy as np
from tensorflow.keras import regularizers
from IPython.display import Image
import time
import os

In [ ]:
!7za x train.zip
!7za x valid.zip
!7za x test.zip
!unzip "final_model.zip" -d "6th_model"

In [ ]:
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode

def take_photo(filename='photo.jpg', quality=0.8):
  js = Javascript('''
    async function takePhoto(quality) {
      const div = document.createElement('div');
      const capture = document.createElement('button');
      capture.textContent = 'Capture';
      div.appendChild(capture);

      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getVideoTracks()[0].stop();
      div.remove();
      return canvas.toDataURL('image/jpeg', quality);
    }
    ''')
  display(js)
  data = eval_js('takePhoto({})'.format(quality))
  binary = b64decode(data.split(',')[1])
  with open(filename, 'wb') as f:
    f.write(binary)
  return filename

In [ ]:
def data_preprocess(
        train_path,
        valid_path,
        size = 75, 
        samplewise_center = True,
        rotation_range = 15,
        zoom_range = 0.1,
        width_shift_range = 0,
        height_shift_range = 0,
        horizontal_flip = True,
        vertical_flip = False):
  
  datagen = ImageDataGenerator(
        samplewise_center = samplewise_center,
        rotation_range = rotation_range,
        zoom_range = zoom_range,
        width_shift_range = width_shift_range,
        height_shift_range = height_shift_range,
        horizontal_flip = horizontal_flip,
        vertical_flip = vertical_flip)

  train_it = datagen.flow_from_directory(train_path, 
                    target_size=(size, size), 
                    color_mode = "rgb",
                    class_mode="categorical", 
                    batch_size = 16)
  
  valid_it = datagen.flow_from_directory(valid_path, 
                    target_size=(size, size), 
                    color_mode = "rgb",
                    class_mode="categorical", 
                    batch_size = 16)
  
  return train_it, valid_it

In [ ]:
def train_model(train_it, valid_it, size = 75):
  model = Sequential()

  model.add(Conv2D(150, (3, 3), strides=1, padding="same", activation="relu", input_shape=(size, size, 3)))
  model.add(BatchNormalization())
  model.add(MaxPool2D((2, 2), strides=2, padding="same"))
    
  model.add(Conv2D(100, (3, 3), strides=1, padding="same", activation="relu"))
  model.add(BatchNormalization())
  model.add(MaxPool2D((2, 2), strides=2, padding="same"))

  model.add(Conv2D(75, (3, 3), strides=1, padding="same", activation="relu"))
  model.add(BatchNormalization())
  model.add(MaxPool2D((2, 2), strides=2, padding="same"))

  model.add(Conv2D(50, (3, 3), strides=1, padding="same", activation="relu"))
  model.add(Dropout(0.35))
  model.add(BatchNormalization())
  model.add(MaxPool2D((2, 2), strides=2, padding="same"))

  model.add(Conv2D(25, (3, 3), strides=1, padding="same", activation="relu"))
  model.add(BatchNormalization())
  model.add(MaxPool2D((2, 2), strides=2, padding="same"))

  model.add(Flatten())
  model.add(Dense(units=512, activation="relu", kernel_regularizer=regularizers.l1_l2(l1=1e-5, l2=1e-4)))
  model.add(Dense(units=6, activation="softmax"))

  model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

  print(model.summary())
  
  history = model.fit(train_it, steps_per_epoch = 6, validation_data=valid_it, validation_steps = 2, epochs = 75)

  return history, model

In [ ]:
def performance(history):
  accuracy = history.history['accuracy']
  loss = history.history['loss']
  val_accuracy = history.history['val_accuracy']
  val_loss = history.history['val_loss']

  epochs = range(1, len(loss) + 1)


  plt.figure(figsize=(10, 10))

  plt.subplot(211)
  plt.plot(epochs, loss, 'b', label = 'Training Loss')
  plt.plot(epochs, val_loss, 'g', label = 'Validation Loss')
  plt.title('Loss')
  plt.xlabel('Epochs')
  plt.ylabel('Loss')
  plt.legend()
  plt.grid()

  plt.subplot(212)
  plt.plot(epochs, accuracy, 'b', label = 'Training Accuracy')
  plt.plot(epochs, val_accuracy, 'g', label = 'Validation Accuracy')
  plt.title('Accuracy')
  plt.xlabel('Epochs')
  plt.ylabel('Accuracy')
  plt.legend()
  plt.grid()

  plt.show()

In [ ]:
def alarm(model, pre_path):
  image = image_utils.load_img(pre_path, target_size=(75, 75))
  image = image_utils.img_to_array(image)
  image = image.reshape(1,75,75,3)
  predictions = model.predict(image)
  cate = np.argmax(predictions,axis = 1)
  if cate[0] == 5:
    try:
      with open('/content/record.txt', 'a', encoding = 'utf-8') as fh:
        fh.write('Warning:intruder at: UTC ' + str(time.ctime()) + '\n')
      filename = take_photo()
      print('Saved to {}'.format(filename))
      display(Image(filename))
    except Exception as err:
      print(str(err))
    print('Freeze!')
    print('==============================================================================')   
  elif cate[0] == 4:
    pass
  else:
    print('Welcome!')
    print('==============================================================================')

In [ ]:
train_it, valid_it = data_preprocess('train', 'valid')
history, model = train_model(train_it, valid_it)
performance(history)
a, b = model.evaluate(valid_it, steps=valid_it.samples/valid_it.batch_size)
print(a)
print(b)

In [ ]:
model.save('final_model') 

In [ ]:
import shutil
try:
  shutil.rmtree('/content/final_model')
except:
  pass

In [ ]:
alarm(model, '/content/test/pre_1.jpg')
alarm(model, '/content/test/pre_2.jpg')